In [2]:
import pandas as pd
from glob import glob
import os
from PIL import Image
import numpy as np
import random

import gc

In [3]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

random.seed(34)

In [4]:
# Скачайте полные данные отсюда https://www.kaggle.com/c/dogs-vs-cats/data (примерно 820мб)
# У вас должно быть две папки train и test. Однако в данном задании мы не будем использвать test выборку
# Функцию для чтения данных мы реализовали за вас

from tqdm.notebook import tqdm

def read_dataset():
    X = np.array([])
    y = np.array([])

    image_paths_list = random.sample(glob(os.path.join('data', 'train', '*.jpg')), 5000)
    np.random.shuffle(image_paths_list)

    for image_path_batch in tqdm(np.split(np.array(image_paths_list), int(len(image_paths_list) / 1000)), desc='Batches'):
        x_batch = []
        y_batch = []
        
        for image_path in tqdm(image_path_batch, desc='Current batch', leave=False):
            image_name_parts = os.path.basename(image_path).split('.')
            label = image_name_parts[0] if len(image_name_parts) == 3 else None
            
            if label:
                y_batch.append(int(label == 'cat'))
            
            x = image.img_to_array(image.load_img(image_path, target_size=(224, 224)))
            x_batch.append(x)

        x_batch = preprocess_input(np.array(x_batch))
        y_batch = np.array(y_batch)
        
        if X.size == 0:
            X = x_batch
            y = y_batch
        else:
            X = np.concatenate([X, x_batch], axis=0)
            y = np.concatenate([y, y_batch], axis=0)
    
    return X, y

Перепишите код read_dataset() таким образом чтобы читать изображения пачками по 1000 и сразу применять предобученный ResNet на них, сохраняя только полученные вектора как множество X. Примените эту функцию на всём датасете 25000 изображений.

In [4]:
# Используйте функцию read_dataset чтобы получить обучающую выборку
features, target = read_dataset()

# Проверьте размерности загруженных данных
print(features.shape, target.shape)


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Current batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Current batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Current batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Current batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Current batch:   0%|          | 0/1000 [00:00<?, ?it/s]

5000 5000


True

In [6]:
# с помощью функции train_test_split поделите выборку на train и test в отношении 70/30
# В качестве признаков используйте эмбеддинги, полученные на предыдущем шаге

features_train, features_test, target_train, target_test = train_test_split(features, target, train_size=0.7, test_size=0.3, random_state=42)

del features
del target
gc.collect()

features_train

(5000, 224, 224, 3) (5000,)


array([[[[-5.5939003e+01, -8.4778999e+01, -8.0680000e+01],
         [-5.5939003e+01, -8.4778999e+01, -8.0680000e+01],
         [-5.5939003e+01, -8.4778999e+01, -8.0680000e+01],
         ...,
         [-7.3939003e+01, -8.8778999e+01, -1.0568000e+02],
         [-7.0939003e+01, -9.7778999e+01, -9.8680000e+01],
         [-7.3939003e+01, -1.0077900e+02, -1.0168000e+02]],

        [[-5.5939003e+01, -8.4778999e+01, -8.0680000e+01],
         [-5.5939003e+01, -8.4778999e+01, -8.0680000e+01],
         [-5.5939003e+01, -8.4778999e+01, -8.0680000e+01],
         ...,
         [-7.6939003e+01, -9.4778999e+01, -1.0468000e+02],
         [-7.1939003e+01, -9.5778999e+01, -9.9680000e+01],
         [-7.4939003e+01, -9.8778999e+01, -1.0268000e+02]],

        [[-5.5939003e+01, -8.5778999e+01, -7.9680000e+01],
         [-5.5939003e+01, -8.5778999e+01, -7.9680000e+01],
         [-5.5939003e+01, -8.5778999e+01, -7.9680000e+01],
         ...,
         [-8.1939003e+01, -1.0577900e+02, -9.6680000e+01],
         [

In [1]:
# Создаём модель для получения эмбеддингов
model = ResNet50(weights='imagenet')

# Используя объект model постройте эмбеддинги для обучающей выборки
%time embds_train = model.predict(features_train)
%time embds_test = model.predict(features_test)

del features_train
del features_test
gc.collect()

NameError: name 'features' is not defined



Создайте объект XGBClassifier со стандартными параметрами

https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier

Обучите его

In [42]:
# Получите предсказания на тестовом и на обучающем множестве
clf = XGBClassifier()
clf.fit(embds_train, target_train)

predicted = clf.predict(embds_test)


/home/ultra/.local/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


ValueError: y should be a 1d array, got an array of shape (1, 1000) instead.

In [ ]:
# С помощью функции accuracy_score оцените результаты
accuracy_score(target_test, predicted)

Почитайте про параметры XGBoost здесь: https://sites.google.com/view/lauraepp/parameters
Сделайте тюнинг гиперпараметров для XGBoost:
1. Подберите максимальный num_iterations при котором скор на валидации близок к максимальному, а скорость обучения ещё приемлимая
2. Для заданного num_iterations подберите подходящий learning rate
3. С помощью библиотеки hyperopt подберите остальные гиперпараметры при фиксимрованных num_iterations и learning_rate. Не подбирайте параметр early_stopping

Оцените важность признаков с помощью поля feature_importances_

Пользуясь списком https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a посмотрите, какие классы оказались наиболее важными (со значениями importance больше 0)
    
Изменяйте параметры, чтобы оставить только действительно важные признаки